In [17]:
using LinearAlgebra, Random, Statistics

In [20]:
# Reference CPU implementation per Numerical Recipes, Press et. al 1992 (§ 2.4)
function tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
    N = length(f)
    ϕ = similar(f)
    γ = similar(f)

    β    = M.d[1]
    ϕ[1] = f[1] / β

    for j = 2:N
        γ[j] = M.du[j-1] / β
        β    = M.d[j] - M.dl[j-1] * γ[j]

        # This should only happen on last element of forward pass for problems
        # with zero eigenvalue. In that case the algorithmn is still stable.
        abs(β) < 1.0e-12 && break

        ϕ[j] = (f[j] - M.dl[j-1] * ϕ[j-1]) / β
    end

    for j = 1:N-1
        k = N-j
        ϕ[k] = ϕ[k] - γ[k+1] * ϕ[k+1]
    end

    return ϕ
end

tridiag (generic function with 1 method)

In [34]:
Nz, Lz = 8, 1
Δz = Lz/Nz

0.125

In [35]:
Rall = rand(MersenneTwister(0), Nz+1)
R = Rall[1:Nz]
Rall[Nz+1] = 0
Rall[1:Nz] = Rall[1:Nz] .- mean(Rall[1:Nz])

R = Rall[1:Nz]

8-element Array{Float64,1}:
  0.4900442230630878 
  0.5767532530121119 
 -0.16903748678063935
 -0.15627443844806   
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [36]:
ud = [1/Δz for k in 2:Nz+1]
ld = [1/Δz for k in 1:Nz]
d  = [-1/Δz, [-2/Δz for k in 2:Nz]..., -1/Δz]
M = Tridiagonal(ld, d, ud)

9×9 Tridiagonal{Float64,Array{Float64,1}}:
 -8.0    8.0     ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     ⋅ 
  8.0  -16.0    8.0     ⋅      ⋅      ⋅      ⋅      ⋅     ⋅ 
   ⋅     8.0  -16.0    8.0     ⋅      ⋅      ⋅      ⋅     ⋅ 
   ⋅      ⋅     8.0  -16.0    8.0     ⋅      ⋅      ⋅     ⋅ 
   ⋅      ⋅      ⋅     8.0  -16.0    8.0     ⋅      ⋅     ⋅ 
   ⋅      ⋅      ⋅      ⋅     8.0  -16.0    8.0     ⋅     ⋅ 
   ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -16.0    8.0    ⋅ 
   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -16.0   8.0
   ⋅      ⋅      ⋅      ⋅      ⋅      ⋅      ⋅     8.0  -8.0

In [37]:
g = Δz .* Rall

9-element Array{Float64,1}:
  0.06125552788288598 
  0.07209415662651399 
 -0.021129685847579918
 -0.0195343048060075  
 -0.006840396947890445
 -0.016265840859050237
 -0.03641270237278686 
 -0.03316675367608497 
  0.0                 

In [38]:
ϕ = tridiag(M, g)[1:Nz]

8-element Array{Float64,1}:
 -0.07351280144284969  
 -0.06585586045748894  
 -0.049187149893813946 
 -0.03515965006108644  
 -0.023573938329109874 
 -0.012843276215619611 
 -0.0041458442095106286
 -3.469446951953614e-18

In [66]:
∇²ϕ = zeros(size(ϕ))

@inline δz(k, f) = @inbounds f[k] - f[k-1]
@inline δz²(k, f) = δz(k+1, f) - δz(k, f)

∇²ϕ[1] = (ϕ[2] - ϕ[1]) / Δz^2  # Enforced via halo regions.
for k in 2:Nz
    ∇²ϕ[k] = δz²(k, ϕ) / Δz^2
end

In [67]:
∇²ϕ

8-element Array{Float64,1}:
  0.49004422306308815
  0.5767532530121113 
 -0.16903748678063923
 -0.15627443844805988
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [68]:
R

8-element Array{Float64,1}:
  0.4900442230630878 
  0.5767532530121119 
 -0.16903748678063935
 -0.15627443844806   
 -0.05472317558312356
 -0.1301267268724019 
 -0.2913016189822949 
 -0.2653340294086798 

In [70]:
∇²ϕ ≈ R

true